In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

from utils.df_loader import load_adult_df, load_compas_df, load_german_df, load_diabetes_df, load_breast_cancer_df
from sklearn.model_selection import train_test_split
from utils.preprocessing import preprocess_df
from utils.models import train_three_models, evaluation_test, save_three_models, load_three_models
from utils.cf_proto import  generate_cf_proto_result, process_result

from utils.save import save_result_as_csv

### Disable TF2 and enable TF1 for alibi.
tf.get_logger().setLevel(40) 
tf.compat.v1.disable_v2_behavior()
tf.keras.backend.clear_session()
pd.options.mode.chained_assignment = None 

print('TF version: ', tf.__version__)
print('Eager execution enabled: ', tf.executing_eagerly()) # False


seed = 123
tf.random.set_seed(seed)
np.random.seed(seed)


TF version:  2.4.0-rc0
Eager execution enabled:  False


In [2]:
#### Select dataset ####

dataset_name = 'adult' # [adult, german, compas]

if dataset_name == 'adult':
    dataset_loading_fn = load_adult_df
elif dataset_name == 'german':
    dataset_loading_fn = load_german_df
elif dataset_name == 'compas':
    dataset_loading_fn = load_compas_df
elif dataset_name == 'diabetes':
    dataset_loading_fn = load_diabetes_df
elif dataset_name == 'breast_cancer':
    dataset_loading_fn = load_breast_cancer_df
else:
    raise Exception("Unsupported dataset")

In [3]:
#### Load datafram info.
df_info = preprocess_df(dataset_loading_fn)

In [4]:
### Seperate to train and test set.
train_df, test_df = train_test_split(df_info.dummy_df, train_size=.8, random_state=seed, shuffle=True)

In [5]:
### Input queries for generating counterfactuals
df_info.df.iloc[test_df[:20].index]

,age,workclass,education,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
20713,55,State-gov,Bachelors,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,15,United-States,<=50K
13495,65,Private,Bachelors,Married-civ-spouse,Transport-moving,Husband,White,Male,0,0,20,United-States,<=50K
12367,29,Private,HS-grad,Never-married,Craft-repair,Not-in-family,White,Male,0,0,40,Guatemala,<=50K
22402,53,State-gov,Doctorate,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,40,United-States,>50K
18338,32,Private,Bachelors,Never-married,Adm-clerical,Not-in-family,Asian-Pac-Islander,Male,0,0,40,United-States,<=50K
1476,38,Private,HS-grad,Divorced,Other-service,Unmarried,White,Female,0,0,48,United-States,<=50K
26209,32,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,43,United-States,>50K
18442,35,Private,Some-college,Married-civ-spouse,Prof-specialty,Husband,Amer-Indian-Eskimo,Male,0,0,40,United-States,<=50K
28842,52,Private,5th-6th,Never-married,Handlers-cleaners,Not-in-family,White,Male,0,0,50,United-States,<=50K
13180,36,Private,Some-college,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,<=50K


In [ ]:
### Get training and testing array.
X_train = np.array(train_df[df_info.ohe_feature_names])
y_train = np.array(train_df[df_info.target_name])
X_test = np.array(test_df[df_info.ohe_feature_names])
y_test = np.array(test_df[df_info.target_name])

In [6]:
### Train models.
# models = train_three_models(X_train, y_train)

### Save models.
# save_three_models(models, dataset_name)

Train on 614 samples
Epoch 1/20
614/614 [==============================] - 0s 119us/sample - loss: 0.6984 - acc: 0.4055
Epoch 2/20
614/614 [==============================] - 0s 28us/sample - loss: 0.6878 - acc: 0.6450
Epoch 3/20
614/614 [==============================] - 0s 18us/sample - loss: 0.6835 - acc: 0.6580
Epoch 4/20
614/614 [==============================] - 0s 22us/sample - loss: 0.6784 - acc: 0.6580
Epoch 5/20
614/614 [==============================] - 0s 24us/sample - loss: 0.6729 - acc: 0.6580
Epoch 6/20
614/614 [==============================] - 0s 37us/sample - loss: 0.6667 - acc: 0.6580
Epoch 7/20
614/614 [==============================] - 0s 16us/sample - loss: 0.6586 - acc: 0.6580
Epoch 8/20
614/614 [==============================] - 0s 33us/sample - loss: 0.6502 - acc: 0.6580
Epoch 9/20
614/614 [==============================] - 0s 26us/sample - loss: 0.6429 - acc: 0.6580
Epoch 10/20
614/614 [==============================] - 0s 15us/sample - loss: 0.6361 - acc: 0.65

In [7]:
### Load models
models = load_three_models(X_train.shape[-1], dataset_name)

/Users/jrhs/miniforge3/envs/tf_mac/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


In [8]:
### Print out accuracy on testset.
evaluation_test(models, X_test, y_test)

DT: [0.7727] | RF [0.7987] | NN [0.7662]


# Alibi

## 1. Counterfactual Prototype

In [9]:
### Setting up the CF generating amount.
num_instances = 5
num_cf_per_instance = 1

In [10]:
### Generate CF
results = generate_cf_proto_result(df_info, train_df, models, num_instances, num_cf_per_instance, X_train, X_test, y_test, max_iters=500)
result_dfs = process_result(results, df_info)

Finding counterfactual for dt
instance 0
CF 0
Found CF
instance 1
CF 0
Found CF
instance 2
CF 0
Found CF
instance 3
CF 0
CF not found
instance 4
CF 0
CF not found
Finding counterfactual for rfc
instance 0
CF 0
Found CF
instance 1
CF 0
CF not found
instance 2
CF 0
Found CF
instance 3
CF 0
CF not found
instance 4
CF 0
CF not found
Finding counterfactual for nn
instance 0
CF 0
Found CF
instance 1
CF 0
CF not found
instance 2
CF 0
CF not found
instance 3
CF 0
CF not found
instance 4
CF 0
CF not found


In [11]:
### Save result as file.
save_result_as_csv("proto", dataset_name, result_dfs)

Result has been saved to ./results/proto_diabetes
